# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |  Regina Tamayo León |
| **Fecha**      | 8 feb 2026  |
| **Expediente** |757857   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

¿Qué puedes observar al ajustar los valores de `alpha`? 

Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.



**Sin penalización**

In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import LinearRegression

df = pd.read_csv('Advertising.csv', index_col=0)

X = np.array(df[['TV', 'radio', 'newspaper']])
y = np.array(df['sales'])

lr = LinearRegression()
lr.fit(X, y)

print("Modelo 1: Sin Penalización")
print("beta_0 (Intercepto):", lr.intercept_)
print("Coeficientes (beta_1, beta_2, beta_3):", lr.coef_)
print("R^2 score:", lr.score(X, y))
import matplotlib.pyplot as plt


Modelo 1: Sin Penalización
beta_0 (Intercepto): 2.938889369459403
Coeficientes (beta_1, beta_2, beta_3): [ 0.04576465  0.18853002 -0.00103749]
R^2 score: 0.8972106381789522


**Con penalización L2**

In [11]:
from sklearn.linear_model import Ridge
#alpha=100
ridge = Ridge(alpha=100)
ridge.fit(X, y)

print("Modelo 2: Ridge (L2)")
print("beta_0:", ridge.intercept_)
print("Coeficientes:", ridge.coef_)
print("R^2 score:", ridge.score(X, y))

Modelo 2: Ridge (L2)
beta_0: 2.9466816422932336
Coeficientes: [ 0.04576446  0.18803935 -0.00091803]
R^2 score: 0.8972089327944492


los coeficientes son casi iguales a la regresión lineal, pero ligeramente más contraídos (especialmente el de Radio bajó de 0.1885 a 0.1880).

**Con penalización L1**

In [10]:
from sklearn.linear_model import Lasso
#alpha=1.0
lasso = Lasso(alpha=1.0)
lasso.fit(X, y)

print("Modelo 3: Lasso (L1)")
print("beta_0:", lasso.intercept_)
print("Coeficientes:", lasso.coef_)
print("R^2 score:", lasso.score(X, y))

Modelo 3: Lasso (L1)
beta_0: 3.040215583480375
Coeficientes: [0.04566142 0.1834644  0.        ]
R^2 score: 0.8970235728389688


el tercer coeficiente (Newspaper) se vuelve 0.0. Lasso decidió que invertir en periódico no tiene un impacto real en las ventas cuando ya existe TV y Radio en el modelo

**¿Qué puedes observar al ajustar los valores de alpha?**

En Ridge (L2): Los coeficientes se reducen gradualmente pero nunca mueren del todo. El R^2 se mantiene muy estable. 
En Lasso (L1): Ayuda a la selección de variables. Si se sube mucho el alpha, variables como newspaper desaparecen del modelo (coeficiente 0), simplificándolo. 

Ridge (L2): A medida que aumentas alpha, los coeficientes se reducen gradualmente hacia cero, pero es muy difícil que lleguen a ser exactamente cero. Por ejemplo, con alpha=1000, el coeficiente de newspaper cambió de signo y se acercó a cero, pero sigue presente.Lasso (L1): Es mucho más drástico. Con un alpha pequeño (como 1.0), Lasso eliminó por completo la variable newspaper igualando su coeficiente a 0.0. Esto demuestra que Lasso funciona como un método de selección de variables.

El valor de R^2 disminuye ligeramente a medida que aumenta la penalización. Esto sucede porque estamos agregando sesgo al modelo a propósito para reducir su varianza.
La caída en el R^2 es mínima (pasa de 0.8972 a 0.8970), lo que indica que podemos tener un modelo más simple (con menos variables en el caso de Lasso) sin perder prácticamente nada de capacidad predictiva.